In [18]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

In [19]:
# -- years being used for evaluation
Years = '2014'

# -- spatial key being used for calculation
SpatialKey = 'ZIPCODE'

# -- input path
InputPath = '../../shapeData/Dataframes/'

# -- Census Park Breakdown input file
CensusParkInputFile = '../../shapeData/Tables/AllSites_MasterPropID_CT.xlsx'

# -- output path
OutputPath = '../../shapeData/Tables/'

In [20]:
# -- read the park quality file on a year basis 
try:
    yearDF
    
except: 
    # -- for each year of analysis, merge files into 1
    
    # -- clear df first so block can be rerun in notebook
    yearDF = pd.DataFrame({})
    for Year in Years.split(','):
        print "reading %s_Inspections.csv..." % (Year)
        df = pd.read_csv(InputPath + '%s_Inspections.csv' % (Year), index_col=0, sep='\t')
        try:
            yearDF = yearDF.append(df, ignore_index=True)
        except:
            yearDF = df

    # -- append resorts columns.  reinstate column order
    yearDF = yearDF.reindex_axis(df.columns, axis=1)

In [21]:
# -- read the park listing breakdowns by census tract
try:
    censusParkDF
except:
    if SpatialKey == 'GEOID':
        print "reading census park info file..."
        censusParkDF = pd.read_excel(CensusParkInputFile)

In [22]:
# -- read the Amenity files 
try:
    amenityFlagFilter
except:
    print "reading Amenity Files"
    amenityFlagFilter = pd.read_excel(OutputPath + "PIP_Inventory_CUSP.xlsx")
    amenityCompleteList = pd.read_excel(OutputPath + "PIP_Inventory_Client.xlsx")

In [23]:
# -- categories
categories = [u'Athletic Fields', u'Benches',
       u'Fences', u'Glass', u'Graffiti', u'Horticultural Areas', u'Ice',
       u'Lawns', u'Litter', u'Paved Surfaces', u'Play Equipment',
       u'Safety Surface', u'Sidewalks', u'Trails', u'Trees', u'Water Bodies',
       u'Weeds']

In [24]:
# -- calculate inspection scores [ratio]
yearDF['Ratings Ratio'] = 1 - yearDF[categories].mean(axis=1)

In [25]:
# -- build attribute list to capture from inspection reports.  
# --   If Spatial included, grab it.  If not, its GEOID which is in a different file
yearDFAttr = ['Prop ID', 'PID_base', 'Category']
if SpatialKey in yearDF.columns:
    yearDFAttr.append(SpatialKey)

# -- seed final park stats DF
byParkID = yearDF[yearDFAttr].groupby(['Prop ID'], as_index = False)
parkStats = byParkID.first()


# -- average Park Score and Acres
byPark = yearDF.groupby(['Prop ID'], as_index = False)
parkAvgScore = byPark['Ratings Ratio','Adjusted Acres'].mean()
parkAvgScore.rename(columns={'Ratings Ratio':'Score'}, inplace=True)
parkAvgScore.rename(columns={'Adjusted Acres':'Acres'}, inplace=True)
parkStats = pd.merge(parkStats, parkAvgScore, on='Prop ID', how='left')

# -- Utilize new park table which has parks broken up by census tracts if used!
if SpatialKey == 'GEOID':
    parkStats = pd.merge(censusParkDF[['Prop ID', 'GEOID', 'Fractional Acres']], parkStats, on = 'Prop ID', how = 'left')
    
    # -- Replace acerage with census breakup acreage
    parkStats['Acres'] = parkStats['Fractional Acres']
    parkStats.drop('Fractional Acres', axis=1, inplace=True)

# -- calculate weighted Score for park
parkStats['Score * Acres'] = parkStats['Score'] * parkStats['Acres']

In [26]:
# -- seed district_category stats DF to calculate the mean Ameneties in each zipcode
byCategory = parksWithAmenities.groupby([SpatialKey] + ['Category'], as_index = False)

# -- calculate average amneties in each zipcode <m>
catAmenity = byCategory['m'].mean()
catAmenity.rename(columns={'m':'<m>'}, inplace=True)

In [27]:
# -- Join the amenity average table to the orginal park info table to calculate the equation
amenityEquationDataframe = pd.merge(parksWithAmenities, catAmenity, on=[SpatialKey,'Category'], how='left')

# -- Apply Equation
amenityEquationDataframe['m/<m>'] = amenityEquationDataframe['m']/amenityEquationDataframe['<m>']
amenityEquationDataframe['m/<m>'].replace(np.inf, 0)
amenityEquationDataframe['m/<m>'].fillna(0, inplace=True)
amenityEquationDataframe['1 + m/<m>'] = 1+(amenityEquationDataframe['m/<m>'])
amenityEquationDataframe['W'] = amenityEquationDataframe['Acres']*amenityEquationDataframe['1 + m/<m>']
amenityEquationDataframe['W*Q'] = amenityEquationDataframe['W']*amenityEquationDataframe['Score']

In [28]:
# -- seed district_category stats DF
byCategory = amenityEquationDataframe.groupby([SpatialKey] + ['Category'], as_index = False)
categoryStats = byCategory.first()[[SpatialKey] + ['Category']]

# -- sum WQ and W per category
catSums = byCategory['Acres', 'Score', 'Score * Acres','W*Q', 'W'].sum()
# catSums.rename(columns={'Weighted Score':'Score'}, inplace=True)
categoryStats = pd.merge(categoryStats, catSums, on=[SpatialKey] + ['Category'], how='left')

# -- count parks per category
catCounts = byCategory.size().reset_index().rename(columns={0:'Counts'})
categoryStats = pd.merge(categoryStats, catCounts, on=[SpatialKey] + ['Category'], how='left')

In [29]:
# -- group by zipcode and category
bySpatialKey = categoryStats.groupby(SpatialKey, as_index = False)
SpatialKeySums = bySpatialKey.sum()[[SpatialKey] + ['Counts']]

categoryStats = pd.merge(categoryStats, SpatialKeySums, on=SpatialKey, how = 'left', suffixes = ['_Cat', '_%s' % SpatialKey])

In [30]:
# -- calculate weighted scores for SpatialKey

# -- Normalized score
categoryStats['Non-Weighted Score'] = categoryStats['Score'] / categoryStats['Counts_Cat']
categoryStats['Naive Normalized Score'] = categoryStats['Non-Weighted Score'] * (categoryStats['Counts_Cat'] / categoryStats['Counts_%s' % SpatialKey])

# -- Area Normalized Score
categoryStats['Weighted Score'] = categoryStats['Score * Acres'] / categoryStats['Acres']
categoryStats['Area Normalized Score'] = categoryStats['Weighted Score'] * (categoryStats['Counts_Cat'] / categoryStats['Counts_%s' % SpatialKey])

# -- Area and Amenities weighted score
categoryStats['Equation Score'] = categoryStats['W*Q'] / categoryStats['W']
categoryStats['Amenities & Area Normalized Score'] = categoryStats['Equation Score'] * (categoryStats['Counts_Cat'] / categoryStats['Counts_%s' % SpatialKey])

In [31]:
# -- sum Scores per Zipcode
bySpatialKey = categoryStats.groupby(SpatialKey)
SpatialStats = bySpatialKey \
                    .sum() \
                    .reset_index() \
                    [[SpatialKey] + ['Naive Normalized Score'] + ['Area Normalized Score'] + ['Amenities & Area Normalized Score']]

In [32]:
# -- floor scores at 0 just in case U/S overtake ratios
SpatialStats.loc[SpatialStats['Amenities & Area Normalized Score'] < 0, 'Amenities & Area Normalized Score'] = 0
SpatialStats.loc[SpatialStats['Area Normalized Score'] < 0, 'Area Normalized Score'] = 0

In [ ]:
# -- Write to csv
now = datetime.now()
SpatialStats.to_csv(OutputPath + 'ParkQuality_%s_%s_%s.csv' % (SpatialKey, Years.replace(',','&'), now.strftime('%m-%d-%Y')), sep=',')